**Appendix A: Creating $n$-Qubit toffoli Gates From Scratch**

In [1]:
ancilla = cirq.GridQubit.rect(1, number_qubits-1, 1)
print(ancilla)

NameError: name 'cirq' is not defined

In this implementation, we implemented the $n$-qubit Toffoli with a built-in function in Cirq. However, the actual decomposition of this gate must be done in terms of Toffoli gates, with a qubit ancilla (whcih we defined above).

We start by applying a Toffoli gate to the first two control qubits, with the target as the first ancilla 
qubit. Then, we apply a Toffoli controlled by the third control qubit and the first ancilla qubit, which we 
targetted with the previous Toffoli gate. We continue this process until all control qubits have been utilized 
in Toffoli gates, and then attach a $CNOT$ gate to our final ancilla qubit, targetting the "overall" original 
target qubit of the $n$-qubit Toffoli gate. Our final step is to perform an <i>uncomputation</i> operation, 
which means that we apply our series of Toffoli gates in reverse. This allows us to revert our ancilla back to 
its initial state $|0\rangle^{\otimes n \ - \ 1}$, thus allowing it to be re-used for future computation.

This process may seem a little bit confusing, but here is an example of the generated circuit for the 
$n$-qubit Toffoli gate for the case of $n \ = \ 4$, when this process is implemented in Cirq:

In [ ]:
number_qubits = 4

def apply_n_qubit_tof(ancilla, args):

    if (len(args) == 2):
        yield cirq.CNOT.on(args[0], args[1])

    elif (len(args) == 3):
        yield cirq.CCX.on(args[0], args[1], args[2])

    else:

        yield cirq.CCX.on(args[0], args[1], ancilla[0])
        for k in range(2, len(args)-1):
            yield cirq.CCX(args[k], ancilla[k-2], ancilla[k-1])

        yield cirq.CNOT.on(ancilla[len(args)-3], args[len(args)-1])

        for k in range(len(args)-2, 1, -1):
            yield cirq.CCX(args[k], ancilla[k-2], ancilla[k-1])
        yield cirq.CCX.on(args[0], args[1], ancilla[0])


control = [cirq.GridQubit(0, i) for i in range(0, number_qubits+1)]
circuit = cirq.Circuit()
circuit.append(apply_n_qubit_tof(ancilla, control))
print(circuit)

The reason why I have introduced an extra qubit is because the way that I coded this function takes the last entry 
in the ``args`` list as the target qubit, thus I needed $4$ control qubits, plus $1$ target qubit for a total 
of $5$ qubits in the ``control`` list (even though one of them is actually the target!).